In [1]:
from utils_general import *
from utils_methods import *
import json

# from utils_methods_FedDC import train_FedDC

In [2]:
class config(object):
    def __init__(self):
        self.dataset="cifar10"
        self.device = "cuda"
        self.comm_round=2
        self.lr = 0.01
        self.batch_size = 64
        self.epochs = 1
        self.weight_decay = 1e-3
        self.n_parties = 10
        self.seed = 42
        self.alg="fedavg"
        self.rootdir="./result/bench2/"
        self.datadir="./Folder/"
        self.beta=0.1
        self.model_name = 'cifar10_LeNet' # Model type


args = config()

In [3]:
from utils_libs_wy import get_logger

# def get_logger(logger_path):
#     logging.basicConfig(
#         filename=logger_path,
#         # filename='/home/qinbin/test.log',
#         format='[%(asctime)s] %(levelname)s: %(message)s',
#         datefmt='%m-%d %H:%M', 
#         level=logging.DEBUG, 
#         filemode='w'
#     )

#     logger = logging.getLogger()
#     logger.setLevel(logging.INFO)
#     ch = logging.StreamHandler()
#     logger.addHandler(ch)

#     return logger 

In [4]:
seed = args.seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

In [5]:
# # Dataset initialization
# data_path = args.datadir
# n_client = args.n_parties
# # Dirichlet (0.3)

# data_obj = DatasetObject(dataset=args.dataset.upper(), n_client=n_client, seed=args.seed, unbalanced_sgm=0, rule='Drichlet', rule_arg=args.beta, data_path=data_path)


In [6]:
# Model function
model_func = lambda : client_model(args.model_name)
init_model = model_func()

tag = f"{args.dataset}-{args.model_name}-{args.alg}-N{args.n_parties}-beta{args.beta}-ep{args.epochs}-lr{args.lr}-round{args.comm_round}"
args.exp_dir = os.path.join(args.rootdir, tag) 
args.logdir = os.path.join(args.exp_dir, f"seed{args.seed}")
os.makedirs(args.logdir)

args.argument_path='experiment_arguments-%s.json' % datetime.datetime.now().strftime("%Y-%m-%d-%H:%M-%S")
with open(os.path.join(args.logdir, args.argument_path), 'w') as f:
    json.dump(str(args), f)
device = torch.device(args.device)

log_path= 'experiment_log-%s' % (datetime.datetime.now().strftime("%Y-%m-%d-%H:%M-%S")) + '.log'
logger = get_logger(logger_path=os.path.join(args.logdir, log_path))


print(f'Trying {args.alg}')
train_FedAvg(
    args,
    logger,
    model_func, 
    init_model, 
    overwrite=True, 
)      


Trying fedavg


100%|██████████| 170498071/170498071 [00:13<00:00, 12307719.47it/s]


Extracting ./data/CIFAR10\cifar-10-python.tar.gz to ./data/CIFAR10
Files already downloaded and verified


AttributeError: 'config' object has no attribute 'num_clients'